In [3]:
import pygame
import math


BOARD_SIZE = 11
HEX_RADIUS = 25
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (220, 20, 60)
BLUE = (65, 105, 225)
GREY = (169, 169, 169)
BOARD_COLOR = (245, 222, 179) 
DOT_COLOR_RED = (199, 21, 133)
DOT_COLOR_BLUE = (0, 0, 139)   


PLAYER_RED = 1
PLAYER_BLUE = 2

class HexBoard:
    def __init__(self, size):
        self.size = size
        self.board = [[0] * size for _ in range(size)]
        self.current_player = PLAYER_RED
        self.turn_count = 0
        self.first_move_pos = None

    def place_piece(self, row, col):
        if 0 <= row < self.size and 0 <= col < self.size and self.board[row][col] == 0:
            self.board[row][col] = self.current_player
            if self.turn_count == 0:
                self.first_move_pos = (row, col)
            
            self.switch_player()
            self.turn_count += 1
            return True
        return False

    def perform_swap(self):
        if self.turn_count != 1 or self.first_move_pos is None:
            return False
        
        r, c = self.first_move_pos
        
        self.board[r][c] = PLAYER_BLUE
        
        self.current_player = PLAYER_RED
        
        self.first_move_pos = None
        self.turn_count += 1
        return True

    def switch_player(self):
        self.current_player = PLAYER_BLUE if self.current_player == PLAYER_RED else PLAYER_RED

    def check_win(self):
        for r in range(self.size):
            if self.board[r][0] == PLAYER_RED:
                if self._is_connected(r, 0, PLAYER_RED, set()):
                    return PLAYER_RED

        for c in range(self.size):
            if self.board[0][c] == PLAYER_BLUE:
                if self._is_connected(0, c, PLAYER_BLUE, set()):
                    return PLAYER_BLUE
        return 0 

    def _is_connected(self, r, c, player, visited):

        if (r, c) in visited:
            return False
        visited.add((r, c))

        if player == PLAYER_RED and c == self.size - 1:
            return True
        if player == PLAYER_BLUE and r == self.size - 1:
            return True

        for dr, dc in self._get_neighbors(r, c):
            nr, nc = r + dr, c + dc
            if (0 <= nr < self.size and 0 <= nc < self.size and
                    self.board[nr][nc] == player):
                if self._is_connected(nr, nc, player, visited):
                    return True
        return False

    def _get_neighbors(self, r, c):
        return [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, 1), (1, -1)]

def draw_goal_indicators(surface, size, radius, offset_x, offset_y):
    hex_width = radius * math.sqrt(3)
    hex_height = radius * 2
    dot_radius = 5

    for r in range(size):
        left_hex_center_x = offset_x + r * hex_width / 2
        left_hex_center_y = offset_y + r * hex_height * 0.75
        pygame.draw.circle(surface, DOT_COLOR_RED, (left_hex_center_x - hex_width * 0.7, left_hex_center_y), dot_radius)

        right_hex_center_x = offset_x + (size - 1) * hex_width + r * hex_width / 2
        right_hex_center_y = offset_y + r * hex_height * 0.75
        pygame.draw.circle(surface, DOT_COLOR_RED, (right_hex_center_x + hex_width * 0.7, right_hex_center_y), dot_radius)

    for c in range(size):
        top_hex_center_x = offset_x + c * hex_width
        top_hex_center_y = offset_y
        pygame.draw.circle(surface, DOT_COLOR_BLUE, (top_hex_center_x, top_hex_center_y - hex_height * 0.6), dot_radius)

        bottom_hex_center_x = offset_x + c * hex_width + (size - 1) * hex_width / 2
        bottom_hex_center_y = offset_y + (size - 1) * hex_height * 0.75
        pygame.draw.circle(surface, DOT_COLOR_BLUE, (bottom_hex_center_x, bottom_hex_center_y + hex_height * 0.6), dot_radius)

def draw_board(surface, board_obj, offset_x, offset_y, font):
    surface.fill(BOARD_COLOR)
    size = board_obj.size
    hex_width = HEX_RADIUS * math.sqrt(3)
    hex_height = HEX_RADIUS * 2

    draw_goal_indicators(surface, size, HEX_RADIUS, offset_x, offset_y)

    for r in range(size):
        for c in range(size):
            x = offset_x + c * hex_width + r * hex_width / 2
            y = offset_y + r * hex_height * 0.75

            piece = board_obj.board[r][c]
            color = GREY
            if piece == PLAYER_RED:
                color = RED
            elif piece == PLAYER_BLUE:
                color = BLUE
            
            points = []
            for i in range(6):
                angle_deg = 60 * i - 30
                angle_rad = math.pi / 180 * angle_deg
                points.append((x + HEX_RADIUS * math.cos(angle_rad), y + HEX_RADIUS * math.sin(angle_rad)))
            pygame.draw.polygon(surface, color, points)
            pygame.draw.polygon(surface, BLACK, points, 2)

            if board_obj.turn_count == 1 and (r, c) == board_obj.first_move_pos:
                swap_text_surf = font.render("S", True, WHITE)
                swap_text_rect = swap_text_surf.get_rect(center=(x, y))
                surface.blit(swap_text_surf, swap_text_rect)

def get_hex_at_pos(pos, size, offset_x, offset_y):
    mouse_x, mouse_y = pos
    hex_width = HEX_RADIUS * math.sqrt(3)
    hex_height = HEX_RADIUS * 2

    for r in range(size):
        for c in range(size):
            x = offset_x + c * hex_width + r * hex_width / 2
            y = offset_y + r * hex_height * 0.75
            if math.sqrt((mouse_x - x)**2 + (mouse_y - y)**2) < HEX_RADIUS:
                return r, c
    return None, None

def main():
    pygame.init()

    hex_width = HEX_RADIUS * math.sqrt(3)
    hex_height = HEX_RADIUS * 2
    grid_render_width = (BOARD_SIZE - 1) * hex_width + (BOARD_SIZE - 1) * hex_width / 2 + 2 * HEX_RADIUS
    grid_render_height = (BOARD_SIZE - 1) * hex_height * 0.75 + 2 * HEX_RADIUS
    PADDING = HEX_RADIUS * 2
    TOP_INFO_PANEL_HEIGHT = 50
    WIDTH = int(grid_render_width + PADDING * 2)
    HEIGHT = int(grid_render_height + PADDING * 2 + TOP_INFO_PANEL_HEIGHT)

    board_area_y_start = TOP_INFO_PANEL_HEIGHT
    board_area_height = HEIGHT - TOP_INFO_PANEL_HEIGHT
    grid_top_left_x = (WIDTH - grid_render_width) / 2
    grid_top_left_y = board_area_y_start + (board_area_height - grid_render_height) / 2
    offset_x = grid_top_left_x + HEX_RADIUS
    offset_y = grid_top_left_y + HEX_RADIUS

    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Hex Game (Swap Rule)")
    font = pygame.font.Font(None, 48)
    swap_font = pygame.font.Font(None, 28) 
    clock = pygame.time.Clock()

    board = HexBoard(BOARD_SIZE)
    running = True
    game_over = False
    winner = 0

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.MOUSEBUTTONDOWN and not game_over:
                r, c = get_hex_at_pos(event.pos, board.size, offset_x, offset_y)
                if r is not None:
                    is_swap_move = (board.turn_count == 1 and (r, c) == board.first_move_pos)

                    if is_swap_move:
                        board.perform_swap()
                    else:
                        if board.place_piece(r, c):
                            winner = board.check_win()
                            if winner != 0:
                                game_over = True
        
        draw_board(screen, board, offset_x, offset_y, swap_font)

        pygame.draw.rect(screen, BOARD_COLOR, (0, 0, WIDTH, TOP_INFO_PANEL_HEIGHT))

        player_text_str = f"Player: {'Red' if board.current_player == PLAYER_RED else 'Blue'}"
        player_text_surface = font.render(player_text_str, True, BLACK)
        screen.blit(player_text_surface, (15, 10))
        
        if game_over:
            winner_text = f"{'Red' if winner == PLAYER_RED else 'Blue'} Wins!"
            win_surface = font.render(winner_text, True, BLACK, WHITE)
            text_rect = win_surface.get_rect(center=(WIDTH/2, HEIGHT/2))
            pygame.draw.rect(screen, WHITE, text_rect.inflate(20, 20))
            screen.blit(win_surface, text_rect)

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

if __name__ == '__main__':
    main()